# Введение в обработку естественного языка
## Урок 13. Модель BERT и GPT
Задание 13
Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза

(дополнительно необязательная задача)на выбор взять https://huggingface.co/datasets/sberquad https://huggingface.co/datasets/blinoff/medical_qa_ru_data натренировать любую модель для вопросно ответной системы как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

In [1]:
!pip install transformers sentencepiece datasets -q

In [2]:
import pandas as pd
from datasets import load_dataset

import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer

In [3]:
corpus = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/Tato/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-57cfc8af008bb08a/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

C:\Users\Tato\anaconda3\envs\tensorflow\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tato\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
result_df = []

def paraphrase_check(row_numb):
    
    text1 = corpus['train']['text_1'][row_numb]
    text2 = corpus['train']['text_2'][row_numb]
    
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).numpy()
    
    p = proba[0]
    
    result_df.append([text1, text2, p[1]])
    
    return pd.DataFrame(result_df, columns=['текст 1', 'текст 2', 'вероятность парафраза'])

In [6]:
paraphrase_check(0)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301


In [7]:
paraphrase_check(8)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Путин перед Днем Победы подписал указ о присво...,СК РФ: Официальный сайт Следственного комитета...,0.054684


In [8]:
paraphrase_check(30)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Путин перед Днем Победы подписал указ о присво...,СК РФ: Официальный сайт Следственного комитета...,0.054684
2,"Прокуратура пока не решила, обжаловать ли приг...",Защита Васильевой обжалует приговор суда,0.239415
